## Import utils

In [0]:
%run ../utils/dataframe_utils

In [0]:
%run ../utils/table_utils

## Import libraries and define constants

In [0]:
import re
from pyspark.sql.functions import col, lit, when, to_date

In [0]:
DS_CATALOG = "1_data_sources"
SCHEMA = "world_telecom_suscriptions"

## Read and clean relevant tables

Fixed broadband subscriptions table

In [0]:
raw_broadband_df = spark.table(f"{DS_CATALOG}.{SCHEMA}.ds_fixed_broadband_subscriptions")

In [0]:
display(raw_broadband_df)

Databricks data profile. Run in Databricks to view.

In [0]:
last_update_text = raw_broadband_df.where(col("series_name").like("%Last Update%")).select("series_name").first()[0]

match = re.search(r"\d{2}/\d{2}/\d{4}", last_update_text)

last_update = match.group() if match else None
print(last_update)

In [0]:
last_update_broadband_df = (
    # filter out rows with only null values or source and last update information
    raw_broadband_df.where(col("series_code").isNotNull())
    .withColumn(
        "last_update", to_date(lit(last_update), "MM/dd/yyyy")
    )
)
display(last_update_broadband_df)

In [0]:
decimal_columns = raw_broadband_df.drop("series_name", "series_code", "country_name", "country_code").columns

clean_broadband_df = clean_decimal_columns(
    last_update_broadband_df,
    columns=decimal_columns,
    replace_nulls=["..", "", "NA", "null"],
    decimal_separator=".",
    thousands_separator=","
)
display(clean_broadband_df)

Fixed telephone subscriptions table

In [0]:
raw_telephone_df = spark.table(f"{DS_CATALOG}.{SCHEMA}.ds_fixed_telephone_subscriptions")

display(raw_telephone_df)

In [0]:
last_update_text = raw_telephone_df.where(col("series_name").like("%Last Update%")).select("series_name").first()[0]

match = re.search(r"\d{2}/\d{2}/\d{4}", last_update_text)

last_update = match.group() if match else None
print(last_update)

In [0]:
last_update_telephone_df = (
    # filter out rows with only null values or source and last update information
    raw_telephone_df.where(col("series_code").isNotNull())
    .withColumn(
        "last_update", to_date(lit(last_update), "MM/dd/yyyy")
    )
)
display(last_update_telephone_df)

In [0]:
decimal_columns = raw_telephone_df.drop("series_name", "series_code", "country_name", "country_code").columns

clean_telephone_df = clean_decimal_columns(
    last_update_telephone_df,
    columns=decimal_columns,
    replace_nulls=["..", "", "NA", "null"],
    decimal_separator=".",
    thousands_separator=","
)
display(clean_telephone_df)

## Write tables

In [0]:
table_config = DataProductConfig(
    table_name=clean_broadband_df.select("series_name").first()[0],
    schema=SCHEMA,
    layer="silver"
)

table_config.write_table(clean_broadband_df)

In [0]:
table_config = DataProductConfig(
    table_name=clean_telephone_df.select("series_name").first()[0],
    schema=SCHEMA,
    layer="silver"
)

table_config.write_table(clean_telephone_df)